<a href="https://colab.research.google.com/github/shjoo0407/political_disposition/blob/main/eun/TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd /content/drive/MyDrive/2023/sw공모전

/content/drive/MyDrive/2023/sw공모전


In [ ]:
ls

drive/  sample_data/


In [ ]:
# 형태소 분석기 Mecab 설치
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
import io
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
import numpy as np

### data load

In [ ]:
df_a1 = pd.read_csv('data/노무현_국힘.csv', sep=',')
df_b1 = pd.read_csv('data/이명박_국힘.csv', sep=',')
df_c1 = pd.read_csv('data/박근혜_국힘.csv', sep=',')
df_d1 = pd.read_csv('data/문재인_국힘.csv', sep=',')

In [ ]:
df_a2 = pd.read_csv('data/노무현.csv', sep=',')
df_b2 = pd.read_csv('data/이명박.csv', sep=',')
df_c2 = pd.read_csv('data/박근혜.csv', sep=',')
df_d2 = pd.read_csv('data/문재인.csv', sep=',')

In [ ]:
df1 = pd.concat([df_a1,df_a2])
df2 = pd.concat([df_b1,df_b2])
df3 = pd.concat([df_c1,df_c2])
df4 = pd.concat([df_d1,df_d2])

In [ ]:
df1 = pd.read_csv('data/노무현_국힘.csv', sep=',')
df2 = pd.read_csv('data/이명박_국힘.csv', sep=',')
df3 = pd.read_csv('data/박근혜_국힘.csv', sep=',')
df4 = pd.read_csv('data/문재인_국힘.csv', sep=',')

In [ ]:
ls

NPP_final_0729.csv  노무현.csv  박근혜.csv
TMP_final_0729.csv  문재인.csv  이명박.csv


In [ ]:
import pandas as pd

def data(startY, startQ, endY, endQ, type):
    '''
    startY: 임기시작 년도, startQ: 임기시작 년도의 분기
    endY: 임기종료 년도, endQ: 임기종료 년도의 분기
    '''

    if type == 'all':
      df1 = pd.read_csv('data/NPP_final_0729.csv', sep=',')
      df2 = pd.read_csv('data/TMP_final_0729.csv', sep=',')

      df1['target'] = '국민의힘'
      df2['target'] = '민주당'

      df = pd.concat([df1, df2]).reset_index()
    elif type == 'npp':
      df = pd.read_csv('data/NPP_final_0729.csv', sep=',')
      df['target'] = '국민의힘'
    else:
      df = pd.read_csv('data/TMP_final_0729.csv', sep=',')
      df['target'] = '민주당'


    df = df[(df['year'] > startY) | ((df['year'] == startY) & (df['quarter'] >= startQ))]
    df = df[(df['year'] < endY) | ((df['year'] == endY) & (df['quarter'] <= endQ))]

    df = df.dropna()



    return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 기간 여기 !!!!!
start_year = 2022
start_quarter = 2
end_year = 2023
end_quarter = 3
data_type = all

df = data(start_year, start_quarter, end_year, end_quarter, data_type)
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,content,target
0,‘빗물을 나르는 고속도로’라 불리는 대심도 빗물터널 조금 전에 둘러보고 왔다. 신월...,국민의힘
1,"오늘 한 언론에 보도된 무소속 박완주 의원의 검찰 공소장 안에는, 강력 성범죄자의 ...",국민의힘
2,전국적인 집중호우로 많은 국민들이 희생되고 막대한 재산 피해가 발생했다. 당장 전면...,국민의힘
3,더불어민주당이 윤석열 정부의 ‘시행령 추진’에 본격적으로 제동을 걸고 있다. KBS...,국민의힘
4,"그칠 줄 모르던 비가 잠시 소강상태에 접어들었지만, 폭우가 할퀴고 간 자리에 남겨진...",국민의힘


In [ ]:
df = df2[:]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(df[df['target']=='국민의힘'].shape)
print(df[df['target']=='민주당'].shape)
print(df.shape)

(0, 4)
(12140, 4)
(12140, 4)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### preprocessing

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import re, string
from konlpy.tag import Komoran, Hannanum
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns

In [ ]:
komoran = Komoran()
hannanum = Hannanum()

In [ ]:
#불용어 사전 만들기
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해']]

In [ ]:
# 정규화
def preprocess(text):
    text=text.strip()
    text=re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text=re.sub(r'[^\w\s]', ' ', str(text).strip())
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text


# 명사/영단어 추출, 한글자 제외, 불용어 제거
def final(text):
    n = []
    word = komoran.nouns(text)
    p = komoran.pos(text)
    for pos in p:
      if pos[1] in ['SL']:
        word.append(pos[0])
    for w in word:
      if len(w)>1 and w not in stopwords:
        n.append(w)
    return " ".join(n)

# 최종
def finalpreprocess(text):
  return final(preprocess(text))


In [ ]:
df['content2'] = df['content'].apply(lambda x: finalpreprocess(x))
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,content,target,content2
40452,"정체불명의 해외발 우편물, 국민 불안이 확산되는데 정부는 손놓고 있습니까 3일째 전...",민주당,정체 불명 해외 우편물 국민 불안 확산 정부 전국 곳곳 해외 발송 정체 불명 우편물...
40453,검찰은 조작 수사도 모자라 이제는 진술을 왜곡해 언론에 퍼뜨려 여론을 호도하고 있습...,민주당,검찰 조작 수사 이제 진술 왜곡 언론 여론 호도 최근 이화영 경기도 평화 부지사 그...
40454,윤석열 대통령은 높아지는 군사적 긴장에 언제까지 팔짱만 끼고 있을 겁니까 북한이 오...,민주당,윤석열 대통령 군사 긴장 팔짱 북한 오늘 새벽 서해 순항 미사일 발사 북한 무력 도...
40455,국정 운영 방향을 남 탓으로 정한 정권은 윤석열 정권뿐입니다 “윤 대통령의 국정 운...,민주당,국정 운영 방향 정권 윤석열 정권 대통령 국정 운영 방향 좌파 정권 동안 대한민국 ...
40456,"누구보다 나라를 사랑했던 청년, 명령에 충실했던 병사를 기억하겠습니다 경북 예천에서...",민주당,나라 사랑 청년 명령 충실 병사 기억 경북 예천 실종자 수색 순직 채수 상병 영결식...


In [ ]:
df = df[df['target']=='민주당']
print(df.shape)

# 여기까지
# 여기까지
# !!!!

(3659, 2)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 파일 저장
# content, target, content2, 기간
df.to_csv('대통령_이름으로_파일_저장.csv')

------------------------------

----------------------

### topic modeling

In [ ]:
!pip install pyLDAvis

In [ ]:
from gensim import corpora
from konlpy.tag import Mecab
from tqdm import tqdm
import re
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
import logging
import pickle
import pyLDAvis.gensim
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=10):
    coherence_values = []
    perplexity_values = []
    model_list = []

    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        perplexity_values.append(model.log_perplexity(corpus))


    return model_list, coherence_values, perplexity_values


def find_optimal_number_of_topics(dictionary, corpus, processed_data, step, start, limit):
    model_list, coherence_values, perplexity_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=processed_data, start=start, limit=limit, step=step)

    x = range(start, limit, step)

    # Plot Coherence Score
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(x, coherence_values, label='Coherence')
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence Score")

    # Plot Perplexity Score
    plt.subplot(1, 2, 2)
    plt.plot(x, perplexity_values, label='Perplexity')
    plt.xlabel("Num Topics")
    plt.ylabel("Perplexity Score")

    plt.tight_layout()
    plt.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def topicmodeling(df):
  df['content3'] = df['content2'].str.split(" ")
  df= df.drop(df[df['content3'].apply(lambda x: isinstance(x, float))].index)

  data_word = df['content3']

  # 정수 인코딩과 빈도수 생성
  dictionary = corpora.Dictionary(data_word)

  # 출현빈도가 적거나 자주 등장하는 단어는 제거
  dictionary.filter_extremes(no_below=10, no_above=0.05)
  corpus = [dictionary.doc2bow(text) for text in data_word]
  print('Number of unique tokens: %d' % len(dictionary))
  print('Number of documents: %d' % len(corpus))

  logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

  find_optimal_number_of_topics(dictionary, corpus, data_word, 1, 2, 30)

  return data_word

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim

def topicmodeling2(df, num, file_name):
  # 정수 인코딩과 빈도수 생성
  dictionary = corpora.Dictionary(data_word)

  # 출현빈도가 적거나 자주 등장하는 단어는 제거
  dictionary.filter_extremes(no_below=10, no_above=0.05)
  corpus = [dictionary.doc2bow(text) for text in data_word]
  print('Number of unique tokens: %d' % len(dictionary))
  print('Number of documents: %d' % len(corpus))

  perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
  coherence_logger = CoherenceMetric(corpus=corpus, coherence="u_mass", logger='shell')

  lda_model = LdaModel(corpus, id2word=dictionary, num_topics=num, passes=1, callbacks=[coherence_logger, perplexity_logger])

  # Compute Coherence Score using c_v
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_word, dictionary=dictionary, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score (c_v): ', coherence_lda)

  # Compute Coherence Score using UMass
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_word, dictionary=dictionary, coherence="u_mass")
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score (u_mass): ', coherence_lda)


  num_topics = num
  topic_words_list = []

  for topic_id in range(num_topics):
      topic_words = lda_model.show_topic(topic_id)
      words = [word for word, _ in topic_words]
      topic_words_list.append({'Topic ID': topic_id + 1, 'Words': words})

  keyword_newsdf = pd.DataFrame(topic_words_list, columns=['Topic ID', 'Words'])

  print("~~~~")
  print(keyword_newsdf)

  print("!!!")
  lda_visualization = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
  print("bb")
  pyLDAvis.save_html(lda_visualization, '노무현_민주당.html')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
topicmodeling2(df1[:10], 11, 'A')

In [ ]:
df4['content3'] = df4['content2'].str.split(" ")
df4= df4.drop(df4[df4['content3'].apply(lambda x: isinstance(x, float))].index)
data_word = df4['content3']
print(data_word)

# 정수 인코딩과 빈도수 생성
dictionary = corpora.Dictionary(data_word)

# 출현빈도가 적거나 자주 등장하는 단어는 제거
dictionary.filter_extremes(no_below=10, no_above=0.05)
corpus = [dictionary.doc2bow(text) for text in data_word]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
coherence_logger = CoherenceMetric(corpus=corpus, coherence="u_mass", logger='shell')

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=15, passes=100, callbacks=[coherence_logger, perplexity_logger])

# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_word, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score (c_v): ', coherence_lda)

# Compute Coherence Score using UMass
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_word, dictionary=dictionary, coherence="u_mass")
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score (u_mass): ', coherence_lda)


num_topics=15
topic_words_list = []

for topic_id in range(num_topics):
    topic_words = lda_model.show_topic(topic_id)
    words = [word for word, _ in topic_words]
    print('Topic ID: ', topic_id + 1, 'Words: ', words)
    topic_words_list.append({'Topic ID': topic_id + 1, 'Words': words})

keyword_newsdf = pd.DataFrame(topic_words_list, columns=['Topic ID', 'Words'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0       [오늘, 정책, 사안, 언급, 시작, 지난주, 홍문표, 의원, 낙농, 업계, 관계자...
1       [소득, 주도, 성장, 성적표, 문재인, 정권, 자화자찬, 일색, 국회, 예정, 발...
2       [민주당, 윤석열, 당선, 발목, 잡기, 대통령, 집무실, 이전, 임대차, 폐지, ...
3       [윤석열, 선인, 후보, 시절, 한일, 양국, 발전, 관계, 역사, 인식, 과거, ...
4       [최원, 함장, 부하, 수장, 막말, 민주당, 조상호, 상근, 부대변인, 독립기념관...
                              ...                        
9172    [인정, 박근혜, 대통령, 사면, 구속, 박근혜, 대통령, 기소, 변호인단, 교체,...
9173    [안철수, 대표, 사면, 발언, 해명, 국민의당, 안철수, 대표, 박근혜, 대통령,...
9174    [홍준표, 블랙리스트, 발언, 당장, 사죄, 자유, 한국당, 홍준표, 후보, 박근혜...
9175    [반기문, 특사, 박근혜, 사면, 안철수, 후보, 자신감, 안철수, 후보, 가지, ...
9176    [홍준표, 후보, 완주, 도하, 홍준표, 경남, 지사, 자유, 한국당, 대통령, 후...
Name: content3, Length: 23303, dtype: object
Number of unique tokens: 14127
Number of documents: 23303

Coherence Score (c_v):  0.5768596473835531

Coherence Score (u_mass):  -2.4101460419008904
Topic ID:  1 Words:  ['대구', '광주', '울산', '호남', '경선', '선대위원장', '상임', '공항', '대회', '공천']
Topic ID:  2 Words:  ['추천', '민정', 'KBS', '사장', '사찰', '공영', '비서관', 'MBC', '차관', '감찰

In [ ]:
lda_visualization = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(lda_visualization, '문재인_통합.html')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
